# VQE Runtime

In [1]:
import numpy as np

from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp, OperatorBase
from qiskit import IBMQ, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit_optimization.problems import QuadraticProgram

from utils import get_optimizer, get_objective

In [2]:
# provider = IBMQ.load_account()

In [3]:
backend = Aer.get_backend('qasm_simulator')

In [4]:
backend.name()

'qasm_simulator'

Required input:
- `problem`: Defines the type of problem, e.g. `"hamiltonian"` or `"feature_selection"` as well as the required input.
- `circuit`: the variational circuit whose parameters to optimize (eventually, we may want to allow predefined options such as `"QAOA"` etc.)
- `optimizer`: classical optimizer to be used (e.g. `"SPSA"`, or `"QN-SPSA"`, but could in principle also be an implementation of `Optimizer`)
- `optimizer_params`: meta parameters required by the optimizer (learning rates, initial points, number of iterations, etc.)
- `callback`: callback to process intermediate results
- `run_config` with 

Run config in `options` (backend, shots, error mitigation, ...) --> or part of `params`?

Possible extensions:
- analytic gradient support

In [5]:
# specify objective function to be optimized

# option 1
# 10 samples with 3 features, select the best subset of features to optimize the score
problem = {
    'type': 'feature_selection',
    'model': 'linear_regression',
    'data': np.random.rand(10, 3),
    'target': np.random.rand(10),
    'score': 'mse'    
}

# option 2
# 3-qubit Hamiltonian, find ground state energy
H = PauliSumOp.from_list([('ZIZ', 1.0), ('ZZI', 2.0)])
problem = {  
    'type': 'energy',
    'H': H,
    # possibly auxs ops etc.
}

# specify variational circuit
circuit = RealAmplitudes(3)

# select optimizer
optimizer = 'QN-SPSA'

# define optimizer parameters
optimizer_params = {
    'maxiter': 100,
    'learning_rate': 1e-2,
    'perturbation': 1e-2,
    'spd_bias': 1e-3,
    # other necessary parameters (otherwise use defaults)
}

# define callback
def callback(*args):
    # get access to the iteration parameters during the optimization
    print(args)

# Mock-Runtime

In [6]:
params = {
    'problem': problem,
    'circuit': circuit,  # serializable as Qobj (or in new format)
    'optimizer': optimizer,
    'optimizer_params': optimizer_params,
    'initial_point': 'random',
}

In [7]:
options = {
    'backend_name': backend.name(),
    'shots': 1000,
    'measurement_error_mitigation': False,  # or select type?
    # other possible settings...
}

In [8]:
def provider_runtime_run(program_name, options, params, callback):
    # TODO
    print(f'Program name: {program_name}')
    print(f'Options:\n{options}')
    print(f'Parameters:\n{params}')

    if program_name != 'VQE':
        raise ValueError(f'Unsupported program name {program_name}. Available: VQE')

    # get the optimizer
    optimizer = get_optimizer(params, callback)

    # get the objective function
    objective = get_objective(params, options)

    # verify the initial point
    circuit = params['circuit']
    initial_point = params['initial_point']
    if initial_point == 'random' or initial_point is None:
        initial_point = np.random.random(circuit.num_parameters)
    elif len(initial_point) != circuit.num_parameters:
        raise ValueError('Mismatching number of parameters and initial point dimension.')

    result = optimizer.optimize(circuit.num_parameters, objective, initial_point=initial_point)

    return result

In [9]:
result = provider_runtime_run('VQE', options, params, callback)

epsize': 0.01, 'accepted': True},)
({'x': array([0.45898771, 0.035616  , 0.21342533, 0.22060143, 0.7530388 ,
       0.17330706, 0.97961487, 0.96818581, 0.62949044, 0.36692092,
       0.73562016, 0.60610184]), 'fx': -0.23828125000000008, 'stepsize': 0.01, 'accepted': True},)
({'x': array([0.46331826, 0.03687475, 0.21477926, 0.22162517, 0.74875424,
       0.17292847, 0.97413416, 0.96981354, 0.62947783, 0.36887937,
       0.74019131, 0.60702983]), 'fx': -0.376953125, 'stepsize': 0.01, 'accepted': True},)
({'x': array([0.45986771, 0.03062056, 0.21393029, 0.21768508, 0.74873716,
       0.17173996, 0.97779346, 0.97164857, 0.62999461, 0.36783055,
       0.73769125, 0.60439434]), 'fx': -0.34179687499999983, 'stepsize': 0.009999999999999998, 'accepted': True},)
({'x': array([0.45697236, 0.0274612 , 0.21270289, 0.21596897, 0.75418811,
       0.17408474, 0.98119562, 0.97560949, 0.62893003, 0.3692503 ,
       0.73472568, 0.6027313 ]), 'fx': -0.6386718750000002, 'stepsize': 0.01, 'accepted': True},

In [10]:
print(f'Optimal value: {result[1]}')
print(f'Function evaluations: {result[2]}')

Optimal value: -0.5527343750000003
Function evaluations: 553
